In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import time

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic

# Function to extract keypoints using MediaPipe Holistic
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, face, lh, rh])

# Function to process each video and extract keypoints for each frame
def process_video(video_path, output_path, gloss, video_id):
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    frame_num = 0  # Frame counter

    # Create directory for gloss if it doesn't exist
    gloss_folder = os.path.join(output_path, gloss)
    if not os.path.exists(gloss_folder):
        os.makedirs(gloss_folder)

    # Create directory for the video if it doesn't exist
    video_folder = os.path.join(gloss_folder, video_id)
    if not os.path.exists(video_folder):
        os.makedirs(video_folder)

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Process the frame with MediaPipe
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image_rgb)

            # Extract keypoints
            keypoints = extract_keypoints(results)

            # Save keypoints for each frame using frame number as filename
            npy_file = os.path.join(video_folder, f"{frame_num}.npy")
            np.save(npy_file, keypoints)
            print(f"Saved keypoints for frame {frame_num} (Gloss: {gloss}, Video ID: {video_id}) at {npy_file}")

            frame_num += 1  # Increment frame counter

    cap.release()

# Function to iterate over all videos based on the CSV file
def process_all_videos(csv_path, videos_folder, output_folder):
    # Load CSV file
    data = pd.read_csv(csv_path)

    start_time = time.time()

    for idx, row in data.iterrows():
        gloss = row['gloss']
        video_id = row['video_id']
        video_file = os.path.join(videos_folder, f"{video_id}")  # Assuming video filenames match the video_id

        if os.path.exists(video_file):
            try:
                process_video(video_file, output_folder, gloss, video_id)
            except Exception as e:
                print(f"Error processing {video_file}: {e}")
        else:
            print(f"Video file {video_file} not found.")
        
        # Log progress every 100 videos
        if (idx + 1) % 100 == 0:
            elapsed_time = time.time() - start_time
            print(f"Processed {idx + 1}/{len(data)} videos. Elapsed time: {elapsed_time:.2f} seconds.")
    
    print(f"Finished processing all videos. Total time: {time.time() - start_time:.2f} seconds.")

# Paths to input and output folders
csv_path = 'C:\\Users\\hamed\\Desktop\\note\\ASL_Citizen\\splits\\train.csv'  # Path to CSV file with gloss and video_id
videos_folder = 'C:\\Users\\hamed\\Desktop\\note\\ASL_Citizen\\videos'  # Folder where videos are stored
output_folder = 'C:\\Users\\hamed\\Desktop\\note\\ASL_Citizen\\keypoints'  # Folder to save the keypoints

# Start processing all videos based on CSV
process_all_videos(csv_path, videos_folder, output_folder)


C:\Users\hamed\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Saved keypoints for frame 0 (Gloss: APPLE, Video ID: 15890366051589533-APPLE.mp4) at C:\Users\hamed\Desktop\note\ASL_Citizen\keypoints\APPLE\15890366051589533-APPLE.mp4\0.npy
Saved keypoints for frame 1 (Gloss: APPLE, Video ID: 15890366051589533-APPLE.mp4) at C:\Users\hamed\Desktop\note\ASL_Citizen\keypoints\APPLE\15890366051589533-APPLE.mp4\1.npy
Saved keypoints for frame 2 (Gloss: APPLE, Video ID: 15890366051589533-APPLE.mp4) at C:\Users\hamed\Desktop\note\ASL_Citizen\keypoints\APPLE\15890366051589533-APPLE.mp4\2.npy
Saved keypoints for frame 3 (Gloss: APPLE, Video ID: 15890366051589533-APPLE.mp4) at C:\Users\hamed\Desktop\note\ASL_Citizen\keypoints\APPLE\15890366051589533-APPLE.mp4\3.npy
Saved keypoints for frame 4 (Gloss: APPLE, Video ID: 15890366051589533-APPLE.mp4) at C:\Users\hamed\Desktop\note\ASL_Citizen\keypoints\APPLE\15890366051589533-APPLE.mp4\4.npy
Saved keypoints for frame 5 (Gloss: APPLE, Video ID: 15890366051589533-APPLE.mp4) at C:\Users\hamed\Desktop\note\ASL_Citizen\